In [1]:
import os

import pandas as pd
import numpy as np

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/globalWarming')
print (engine.table_names())

['CO2ConsumptionPerCapita', 'GlobalMeanTemp', 'GlobalMnTemp', 'GlobalPercapitaCO2Emission', 'PerCapitaCO2Consumption']


In [2]:

#################################################
# Database Setup
#################################################
#- Following line is to suppress the following message 
#- warnings.warn('SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  
#- Set it to True to suppress this warning.')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/globalWarming"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
GlobalCO2Emission = Base.classes.GlobalPercapitaCO2Emission
Samples = Base.classes.GlobalMnTemp
Samples_Metadata = Base.classes.GlobalMnTemp

In [3]:


@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")



In [4]:

@app.route("/names")
def names():
    """Return a list of sample names."""

   # # Use Pandas to perform the sql query
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

   # # Return a list of the column names (sample names)
    return jsonify(list(df.columns)[2:])

   # selYear = [
   #     Samples.Year,
   # ]

   # Years = db.session.query(*selYear).all()
   # return jsonify(list(Years))

In [5]:

@app.route("/metadata/<sample>")
def sample_metadata(sample):
    """Return the MetaData for a given sample."""
    sel = [
        Samples_Metadata.Year,
        Samples_Metadata.Jan,
        Samples_Metadata.Feb,
        Samples_Metadata.Mar,
        Samples_Metadata.Apr,
        Samples_Metadata.May,
        Samples_Metadata.Jun,
        Samples_Metadata.Jul,
        Samples_Metadata.Aug,
        Samples_Metadata.Sep,
        Samples_Metadata.Oct,
        Samples_Metadata.Nov,
        Samples_Metadata.Dec,
        Samples_Metadata.DJF,
        Samples_Metadata.MAM,
        Samples_Metadata.JJA,
        Samples_Metadata.SON,
        Samples_Metadata.Jun,
    ]

    #results = db.session.query(*sel).filter(Samples_Metadata.Year == sample).all()
    results = db.session.query(*sel).all()

    # Create a dictionary entry for each row of metadata information
    sample_metadata = {}
    for result in results:
        sample_metadata["Year"] = result[0]
        sample_metadata["Jan"] = result[1]
        sample_metadata["Feb"] = result[2]
        sample_metadata["Mar"] = result[2]
        sample_metadata["Apr"] = result[4]
        sample_metadata["May"] = result[5]
        sample_metadata["Jun"] = result[6]
        sample_metadata["Jul"] = result[7]
        sample_metadata["Aug"] = result[8]
        sample_metadata["Sep"] = result[8]
        sample_metadata["Oct"] = result[10]
        sample_metadata["Nov"] = result[11]
        sample_metadata["Dec"] = result[12]
        sample_metadata["DJF"] = result[13]
        sample_metadata["MAM"] = result[14]
        sample_metadata["JJA"] = result[15]
        sample_metadata["SON"] = result[16]
        sample_metadata["Jun"] = result[17]

    print(sample_metadata)
    return jsonify(sample_metadata)




In [6]:
@app.route("/samples/<sample>")
def samples(sample):
    """Return ``Year`,'Dec', 'DJF', 'MAM', 'JJA', 'SON', 'and `Dec`."""
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df.loc[df['Year'] < 2019, ["Year", sample]]
    sample_data.columns = ['Year', 'tempAnnamolies']
    sample_data['Month']=sample
    
    #sample_data['tempAnnamolies']=sample_data['tempAnnamolies'] * 10
    # Sort by sample
    sample_data.sort_values(by="Year", ascending=False, inplace=True)

    # Format the data to send as json
    data = {
        "Year": sample_data.Year.values.tolist(),
        "tempAnnamolies": sample_data.tempAnnamolies.values.tolist(),
         "Month": sample_data.Month.tolist(),
    }
    print(data)
    return jsonify(data)


In [ ]:


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Aug/2019 00:38:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2019 00:38:29] "GET /static/js/bonus.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Aug/2019 00:38:29] "GET /static/js/app.js HTTP/1.1" 304 -
127.0.0.1 - - [01/Aug/2019 00:38:30] "GET /names HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2019 00:38:30] "GET /metadata/Feb HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2019 00:38:30] "GET /samples/Feb HTTP/1.1" 200 -


{'Year': 2018, 'Jan': 0.77, 'Feb': 0.85, 'Mar': 0.85, 'Apr': 0.87, 'May': 0.81, 'Jun': 0.74, 'Jul': 0.79, 'Aug': 0.73, 'Sep': 0.73, 'Oct': 0.99, 'Nov': 0.78, 'Dec': 0.89, 'DJF': 0.83, 'MAM': 0.86, 'JJA': 0.75, 'SON': 0.84}
{'Year': [2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948, 1947, 1946, 1945, 1944, 1943, 1942, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934, 1933, 1932, 1931, 1930, 1929, 1928, 1927, 1926, 1925, 1924, 1923, 1922, 1921, 1920, 1919, 1918, 1917, 1916, 1915, 1914, 1913, 1912, 1911, 1910, 1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901, 1900, 1899, 1898, 1897, 1896, 1895, 1894, 1893, 1892, 1891,